In [2]:
import numpy as np
import pickle

## Load pairwise distances and generate TSPLIB file: "TSP_instance.tsp"

In [3]:
with open('/data/SS_RNA_seq/Code/pairwise_jensen_shannon_divergence.dat', 'rb') as infile:
    D = pickle.load(infile)    
    Ds = np.rint(D*100000)
    Ds= Ds.astype(int)

In [4]:
A=Ds
zc=np.zeros((np.size(A[:,1]),1), dtype=np.int)
zr=np.zeros((1,np.size(A[:,1])+1), dtype=np.int)
A=np.c_[zc,A]
A=np.r_[zr,A]


In [5]:
f = open("TSP_instance.tsp", "w")
#write header
f.writelines(["NAME : TSP instance"+'\n',
"COMMENT : equivalence class clustering"+'\n',
"TYPE : TSP\n",
"DIMENSION :" + str(np.size(A[:,1]))+'\n',
"EDGE_WEIGHT_TYPE : EXPLICIT"+'\n',
"EDGE_WEIGHT_FORMAT : FULL_MATRIX"+'\n',
"EDGE_WEIGHT_SECTION"+'\n'])
#write edge weights
for i in range(np.size(A[:,1])):
    row_str = ' '.join(['%d' % num for num in A[i,:]])+'\n'
    f.write(row_str)
f.write("EOF")   
f.close()

## Run Concorde to solve TSP_instance.tsp  .... 
## ... and then continue here to extract/save the optimal path:

In [6]:
#read the tsp_solution and get opt path
with open('path/to/concorde/build/TSP/TSP_instance.sol', 'r') as infile:
    lines=infile.readlines()
path=[]
for line in lines[1:]:
    path = path+ [int(s) for s in line[:-2].split(' ')]

#sanity check
dist=np.zeros(3006)
for i in range(3005):
        dist[i]=A[i,i+1]
print "the direct traversal has distance: " + str(dist.sum())
           
dist=np.zeros(3006)
for i in range(3005):
        dist[i]=A[path[i],path[i+1]]
print "the optimal traversal has distance: "+str(dist.sum())    
    
#remove dummy node and make it zero-indexed    
path=path[1:]  
path=path -np.ones(3005)   

#check 
print path
path.max()




the direct traversal has distance: 176858046.0
the optimal traversal has distance: 156370907.0
[  755.   761.   790. ...,  1420.  2624.  2578.]


3004.0

In [7]:
#Save path
pickle.dump( path, open( "TSP_sol_path.dat", "wb" ) )